# Funcs + Imports

In [ ]:
%reload_ext autoreload
%autoreload 2
#-------------------------- Standard Imports --------------------------#
import kdephys as kde
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import acr
from acr.plots import pub, lrg
from acr.utils import NNXR_GRAY, SOM_BLUE
lrg()
plt.style.use('fast')
from scipy import stats
# ---------------------------- EXTRAS --------------------------------#
from kdephys.plot.main import _title, bp_plot
import kdephys.utils.spectral as sp
bands = sp.bands
import matplotlib.animation as animation
from matplotlib.collections import PathCollection
from IPython.display import HTML
import pickle
import cv2
import os
import warnings
warnings.filterwarnings('ignore')
def _plt_style():
    plt.style.use('fast')
    plt.style.use('/home/kdriessen/gh_master/kdephys/kdephys/plot/acr_plots.mplstyle')
    return
pu = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/pub_utils.py', 'pu')
dag = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/data_agg.py', 'dag')

In [ ]:
import os
import subprocess
import shutil

def get_video_framerate(video_path: str) -> float:
    """Gets the framerate of a video using ffprobe."""
    if not shutil.which("ffprobe"):
        print("Error: ffprobe not found. Please install ffmpeg.")
        return None
    
    command = [
        "ffprobe",
        "-v", "error",
        "-select_streams", "v:0",
        "-show_entries", "stream=r_frame_rate",
        "-of", "default=noprint_wrappers=1:nokey=1",
        video_path,
    ]
    result = subprocess.run(command, capture_output=True, text=True)
    if result.returncode != 0:
        print(f"Error getting framerate: {result.stderr}")
        return None
    
    try:
        num, den = map(int, result.stdout.strip().split('/'))
        return num / den
    except (ValueError, ZeroDivisionError) as e:
        print(f"Could not parse framerate from ffprobe output: '{result.stdout.strip()}' due to {e}")
        return None

def slice_vid_gem(video_path: str, start_time: float, end_time: float, output_path: str):
    """
    Slices a video file from start_time to end_time using ffmpeg and saves it to output_path.
    This version uses libx264 for better compatibility.

    Args:
        video_path (str): Path to the input video file.
        start_time (float): Start time in seconds.
        end_time (float): End time in seconds.
        output_path (str): Path to save the sliced video.
    """
    if not os.path.exists(video_path):
        print(f"Error: Video file not found at {video_path}")
        return

    if not shutil.which("ffmpeg"):
        print("Error: ffmpeg not found. Please install ffmpeg.")
        return
        
    fps = get_video_framerate(video_path)
    if fps is None:
        return
        
    duration = end_time - start_time
    total_frames_in_slice = int(duration * fps)

    print(f"Original framerate: {fps:.2f} FPS")
    print(f"Total frames in new slice: {total_frames_in_slice}")
    
    if os.path.exists(output_path):
        os.remove(output_path)

    command = [
        "ffmpeg",
        "-i", video_path,
        "-ss", str(start_time),
        "-to", str(end_time),
        "-c:v", "libx264",
        "-c:a", "aac",
        "-preset", "fast",
        "-crf", "0",
        output_path,
    ]
    
    print(f"\nRunning ffmpeg command:\n{' '.join(command)}\n")
    
    result = subprocess.run(command, capture_output=True, text=True)

    if result.returncode != 0:
        print(f"Error slicing video with ffmpeg:")
        print(result.stderr)
    else:
        print(f"Sliced video saved to {output_path}")

# Check + Animate

times

OFF_induction --> 16854

SD-wake --> 92

BL-sleep --> 640

In [ ]:
subject = 'ACR_25'
exp = 'swi'

#---------------- Adjust Parameters Here -----------------# 
stores = ['NNXo', 'NNXr']
rel_state='NREM'
#---------------------------------------------------------#
notebook_save_root = '/Volumes/opto_loc/Data/ACR_PROJECT_MATERIALS/plots_presentations_etc/paper_figures/animations/raw_data'

In [ ]:
rec = 'swi-bl'
root_path = acr.io.acr_path(subject, rec)
for f in os.listdir(root_path):
    if f.endswith('.avi'):
        path_to_video = os.path.join(root_path, f)
path_to_video

In [ ]:
video_name = 'bl_sleep'
output_path = f'{notebook_save_root}/{video_name}.mp4'

In [ ]:
slice_vid_gem(path_to_video, start_time=640, end_time=650, output_path=output_path)

In [ ]:
# ----------------------------------------- subject_info + Hypno -----------------------------------------
h = acr.io.load_hypno_full_exp(subject, exp)
hd = acr.hypnogram_utils.create_acr_hyp_dict(subject, exp)
si = acr.info_pipeline.load_subject_info(subject)
sort_ids = [f'{exp}-{store}' for store in stores]
recordings = acr.info_pipeline.get_exp_recs(subject, exp)
stim_store = si['stim-exps'][exp][0]

sd_true_start, stim_start, stim_end, rebound_start, full_exp_start = acr.info_pipeline.get_sd_exp_landmarks(subject, exp, update=True)


mua = acr.mua.load_concat_peaks_df(subject, exp)
fp = acr.io.load_concat_raw_data(subject, recordings)
pon, poff = acr.stim.get_individual_pulse_times(subject, exp)

bindf = dag.load_binned_off_df(subject, exp, count_res='1ms', channel_threshold=15)

In [ ]:
anim_name = 'OFF_induction'
probe_to_animate = 'NNXo'
start_time = 16854
duration = '10s'
rec = 'swi'
st = acr.utils.dt_from_tdt(subject, rec, start_time)
col = SOM_BLUE if probe_to_animate == 'NNXo' else NNXR_GRAY

In [ ]:
# check raster plot
m2p = mua.prb(probe_to_animate).ts(st, st+pd.Timedelta(duration)).to_pandas()
pon2p = pon[(pon>=st)&(pon<st+pd.Timedelta(duration))]
poff2p = poff[(poff>st)&(poff<=st+pd.Timedelta(duration))]

f, ax = simple_raster(m2p, pon2p, poff2p, color=col, figsize=(28, 4))
shade_oodf(ax, bindf.prb(probe_to_animate).oots(st, st+pd.Timedelta(duration)), off_color='red', on_color='green', alpha=0.5, single_ax=True)

In [ ]:
#limits = pickle.load(open(f'{subject}_{exp}_nnxo_lims.pkl', 'rb'))
ylim = (-1340, 1260)
limits = [ylim for i in range(16)]

In [ ]:
# PLOT THE TRACES TO BE ANIMATED
plt.rcParams['ytick.left'] = False
fp2p = fp.prb(probe_to_animate).ts(st, st+pd.Timedelta(duration))

data = fp2p.data.T
times = fp2p.datetime.data
f, ax =quick_trace_plot(data, times, pon2p, poff2p)
for a in ax:
    a.set_xlim(times[0], times[-1])
    a.set_ylim(ylim[0], ylim[1])

In [ ]:
fp2p = fp.prb(probe_to_animate).ts(st, st+pd.Timedelta(duration))
data = fp2p.data.T
times = fp2p.datetime.data
pon2p = pon[(pon>=st)&(pon<st+pd.Timedelta(duration))]
poff2p = poff[(poff>st)&(poff<=st+pd.Timedelta(duration))]


# Create the animation
trace_anim = animated_trace_numpy(
    data=data,
    times=times,
    stim_starts=pon2p,
    stim_ends=poff2p,
    color=col,
    fps=30,
    duration=int(duration.split('s')[0]),
    save_path=f'{notebook_save_root}/{anim_name}--LFP--{probe_to_animate}.mov',
    ylims=limits
)

# To display in the notebook instead of saving
# HTML(trace_anim.to_jshtml())

In [ ]:
# select data for raster plot
m2p = mua.prb(probe_to_animate).ts(st, st+pd.Timedelta(duration)).to_pandas()
pon2p = pon[(pon>=st)&(pon<st+pd.Timedelta(duration))]
poff2p = poff[(poff>st)&(poff<=st+pd.Timedelta(duration))]
# try using the total duraiton of the field potential data
timespd = pd.to_datetime(times)
tr = (timespd.max() - timespd.min()).total_seconds()

# Create the animated raster plot
# Generate the animation - adjust parameters as needed
anim = animated_raster(
    data=m2p, 
    stim_start=pon2p, 
    stim_ends=poff2p, 
    color=col, 
    figsize=(14, 2),
    fps=30,
    duration=int(duration.split('s')[0]),  # Match the 7s data duration
    save_path=f'{notebook_save_root}/{anim_name}--RASTER--{probe_to_animate}.mov',  # Comment out to preview instead of saving
    trange=tr
)

# Display the animation in the notebook
# HTML(anim.to_jshtml())

# Uncomment the above line to display the animation in the notebook
# Or use the save_path parameter to save to a file

In [ ]:
probe_to_animate = 'NNXr'
col = SOM_BLUE if probe_to_animate == 'NNXo' else NNXR_GRAY

In [ ]:
fp2p = fp.prb(probe_to_animate).ts(st, st+pd.Timedelta(duration))
data = fp2p.data.T
times = fp2p.datetime.data
pon2p = pon[(pon>=st)&(pon<st+pd.Timedelta(duration))]
poff2p = poff[(poff>st)&(poff<=st+pd.Timedelta(duration))]


# Create the animation
trace_anim = animated_trace_numpy(
    data=data,
    times=times,
    stim_starts=pon2p,
    stim_ends=poff2p,
    color=col,
    fps=30,
    duration=int(duration.split('s')[0]),
    save_path=f'{notebook_save_root}/{anim_name}--LFP--{probe_to_animate}.mov',
    ylims=limits
)

# To display in the notebook instead of saving
# HTML(trace_anim.to_jshtml())

In [ ]:
# select data for raster plot
m2p = mua.prb(probe_to_animate).ts(st, st+pd.Timedelta(duration)).to_pandas()
pon2p = pon[(pon>=st)&(pon<st+pd.Timedelta(duration))]
poff2p = poff[(poff>st)&(poff<=st+pd.Timedelta(duration))]
# try using the total duraiton of the field potential data
timespd = pd.to_datetime(times)
tr = (timespd.max() - timespd.min()).total_seconds()

# Create the animated raster plot
# Generate the animation - adjust parameters as needed
anim = animated_raster(
    data=m2p, 
    stim_start=pon2p, 
    stim_ends=poff2p, 
    color=col, 
    figsize=(14, 2),
    fps=30,
    duration=int(duration.split('s')[0]),  # Match the 7s data duration
    save_path=f'{notebook_save_root}/{anim_name}--RASTER--{probe_to_animate}.mov',  # Comment out to preview instead of saving
    trange=tr
)

# Display the animation in the notebook
# HTML(anim.to_jshtml())

# Uncomment the above line to display the animation in the notebook
# Or use the save_path parameter to save to a file

# Generates the pickle files with ylim values

In [ ]:
def animated_trace_numpy(data, times, stim_starts, stim_ends, color=SOM_BLUE, 
                         hspace=-0.6, figsize=(28, 10), fps=30, duration=7, 
                         save_path=None, ylims=None):
    """
    Create an animated plot of neural traces with stimulus overlays.
    
    Parameters
    ----------
    data : np.ndarray
        Raw data to plot, of shape (n_channels, n_samples)
    times : np.ndarray
        Times of the data, of shape (n_samples,)
    stim_starts : list-like
        Start times of stimulations
    stim_ends : list-like
        End times of stimulations
    color : str, optional
        Color of the traces, by default SOM_BLUE
    hspace : float, optional
        Space between traces, by default -0.6
    figsize : tuple, optional
        Size of the figure, by default (28, 10)
    fps : int, optional
        Frames per second for the animation, by default 30
    duration : float, optional
        Duration of the animation in seconds, by default 7
    save_path : str, optional
        Path to save the animation, by default None
        
    Returns
    -------
    anim : matplotlib.animation.Animation
        The animation object
    """
    # Set up figure and axes
    f, axes = plt.subplots(data.shape[0], 1, figsize=figsize)
    if data.shape[0] == 1:
        axes = [axes]  # Make iterable if only one channel
    
    plt.subplots_adjust(hspace=hspace)
    
    # Style settings
    plt.rcParams['ytick.left'] = False
    plt.rcParams['axes.spines.bottom'] = False
    plt.rcParams['axes.spines.left'] = False
    plt.rcParams['axes.spines.right'] = False
    plt.rcParams['axes.spines.top'] = False
    
    # Convert times to numeric if they are datetime objects
    if isinstance(times[0], pd.Timestamp):
        start_time = times[0]
        numeric_times = np.array([(t - start_time).total_seconds() for t in times])
        stim_starts_sec = np.array([(t - start_time).total_seconds() for t in stim_starts])
        stim_ends_sec = np.array([(t - start_time).total_seconds() for t in stim_ends])
    else:
        # Assuming they're already numeric
        numeric_times = times
        start_time = times[0]
        stim_starts_sec = np.array([t for t in stim_starts])
        stim_ends_sec = np.array([t for t in stim_ends])
    
    # Time range for the animation
    time_range = numeric_times[-1] - numeric_times[0]
    
    # Calculate y-axis limits for each channel
    nnxo_lims = []
    y_padding = .001  # Add 10% padding to y limits
    for i, ax in enumerate(axes):
        channel_data = data[i, :]
        y_min = np.min(channel_data)
        y_max = np.max(channel_data)
        y_range = y_max - y_min
        nnxo_lims.append((y_min, y_max))
        # Set y limits with padding
        ax.set_ylim(y_min - y_range * y_padding, y_max + y_range * y_padding)
        
        # Set x limits
        if isinstance(times[0], pd.Timestamp):
            ax.set_xlim(times[0], times[-1])
        else:
            ax.set_xlim(numeric_times[0], numeric_times[-1])
            
        ax.set_yticks([])
    
    # Define the pickle file path
    pickle_path = f"{subject}_{exp}_nnxo_lims.pkl"
    
    # Save the nnxo_lims list to the pickle file
    with open(pickle_path, 'wb') as f:
        pickle.dump(nnxo_lims, f)
    
    # Create empty line objects for each channel
    lines = []
    for i in range(data.shape[0]):
        line, = axes[i].plot([], [], color=color, lw=1)
        lines.append(line)
    
    # Create empty spans to track added spans
    spans = []
    
    # Precompute stimulus span coordinates for all channels
    stim_span_info = []
    for start, end in zip(stim_starts, stim_ends):
        stim_span_info.append((start, end))
    
    # Number of frames for the animation
    n_frames = int(fps * duration)
    
    def init():
        # Initialize empty lines
        for line in lines:
            line.set_data([], [])
        return lines
    
    def update(frame):
        # Calculate current time based on frame number
        print(frame)
        current_progress = frame / n_frames
        current_idx = min(int(current_progress * len(numeric_times)), len(numeric_times) - 1)
        
        # Update each line with data up to the current time
        for i, line in enumerate(lines):
            if isinstance(times[0], pd.Timestamp):
                line.set_data(times[:current_idx+1], data[i, :current_idx+1])
            else:
                line.set_data(numeric_times[:current_idx+1], data[i, :current_idx+1])
        
        # Add stimulus spans that should be visible by this time
        current_time = numeric_times[current_idx]
        artists = lines.copy()
        
        # Check which stimuli should be visible and add them if not already added
        for i, (start, end) in enumerate(zip(stim_starts_sec, stim_ends_sec)):
            # Create a unique identifier for this span
            span_id = f"span_{i}"
            
            # If current time has passed the start of this stimulus and we haven't added it yet
            if current_time >= start and not any(s[0] == span_id for s in spans):
                # Add the span to all axes
                for ax_idx, ax in enumerate(axes):
                    # Create the span
                    span = ax.axvspan(
                        stim_span_info[i][0],  # Use original time format
                        stim_span_info[i][1],  # Use original time format
                        color='cornflowerblue',
                        ymin=0.325,
                        ymax=0.712,
                        alpha=0.5
                    )
                    
                    # Track this span with its ID
                    spans.append((span_id, span))
                    artists.append(span)
        
        # Add all existing spans to the artists list
        for _, span in spans:
            if span not in artists:
                artists.append(span)
        
        return artists
    
    # Create the animation
    anim = animation.FuncAnimation(
        f, update, frames=n_frames, init_func=init, blit=True, interval=1000/fps
    )
    
    if save_path:
        # Save as video file
        anim.save(save_path, writer='ffmpeg', fps=fps)
        plt.close(f)
    
    return anim
